# Objetivo 3

In [ ]:
!pip install catboost
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, accuracy_score
import matplotlib.pyplot as plt

df = pd.read_csv('')

In [47]:
#### 2020
##
# Criação dos df de teste
test_20 = df[df.ANO_SEM_NOT < 2023].sample(frac=0.2, random_state=42)
y_test_20 = test_20['DESFECHO']
X_test_20 = test_20.drop(columns=['DESFECHO', 'ANO_SEM_NOT'])
df = df.drop(test_20.index)
# Criação dos df de treino
train_20 = df[df.ANO_SEM_NOT < 2023]
y_train_20 = train_20['DESFECHO']
X_train_20 = train_20.drop(columns=['DESFECHO', 'ANO_SEM_NOT'])
#
#
#### 2021
##
# Criação dos df de teste
test_21 = df[df.ANO_SEM_NOT > 2122].sample(frac=0.2, random_state=42)
y_test_21 = test_21['DESFECHO']
X_test_21 = test_21.drop(columns=['DESFECHO', 'ANO_SEM_NOT'])
df = df.drop(test_21.index)
#
# Criação dos df de treino
#
train_21 = df[df.ANO_SEM_NOT > 2122]
y_train_21 = train_21['DESFECHO']
X_train_21 = train_21.drop(columns=['DESFECHO', 'ANO_SEM_NOT'])
#
##
#### GERAL
# Criação dos df de teste para intervalo 2023-2122                           
test_geral = df[(df.ANO_SEM_NOT >= 2023) & (df.ANO_SEM_NOT <= 2122)].sample(frac=0.2, random_state=42) 
df = df.drop(test_geral.index)
test_geral = test_geral.append(test_20,ignore_index=True)
test_geral = test_geral.append(test_21,ignore_index=True)
y_test_geral = test_geral['DESFECHO']
X_test_geral = test_geral.drop(columns=['DESFECHO', 'ANO_SEM_NOT'])
#
# Criação dos df de treino
#
y_train_geral = df['DESFECHO']
X_train_geral = df.drop(columns=['DESFECHO', 'ANO_SEM_NOT'])

In [48]:
y_test_20 = y_test_20.reset_index(drop=True)
X_test_20 = X_test_20.reset_index(drop=True)
y_train_20 = y_train_20.reset_index(drop=True)
X_train_20 = X_train_20.reset_index(drop=True)

y_test_21 = y_test_21.reset_index(drop=True)
X_test_21 = X_test_21.reset_index(drop=True)
y_train_21 = y_train_21.reset_index(drop=True)
X_train_21 = X_train_21.reset_index(drop=True)

y_test_geral = y_test_geral.reset_index(drop=True)
X_test_geral = X_test_geral.reset_index(drop=True)
y_train_geral = y_train_geral.reset_index(drop=True)
X_train_geral = X_train_geral.reset_index(drop=True)

In [50]:
def ic(y_true, y_pred_proba):
    n_bootstraps = 1000
    rng_seed = 42
    bootstrapped_scores = []
    rng = np.random.RandomState(rng_seed)

    for i in range(n_bootstraps):
        # Reamostragem aleatória com reposição dos índices dos dados
        indices = rng.randint(0, len(y_pred_proba), len(y_pred_proba))
        
        # Se a amostra só tem uma classe, pule esta iteração
        if len(np.unique(y_true[indices])) < 2:
            single_class_samples += 1
            continue

        score = roc_auc_score(y_true[indices], y_pred_proba[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
    return confidence_lower, confidence_upper


In [ ]:
### MODELO GERAL
#
# Treino do modelo
model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=5, learning_rate=0.1, verbose=0)
model.fit(X_train_geral, y_train_geral)
#
# Calculo auc para população GERAL
#
y_pred_proba = model.predict_proba(X_test_geral)[:, 1]
fpr, tpr, _ = roc_curve(y_test_geral, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo GERAL com pop Geral: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_geral, y_pred_proba)
print("IC AUC do Modelo GERAL com pop Geral: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#
# Calculo auc para população 20
#
y_pred_proba = model.predict_proba(X_test_20)[:, 1]
fpr, tpr, _ = roc_curve(y_test_20, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo GERAL com pop 20: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_20, y_pred_proba)
print("IC AUC do Modelo GERAL com pop 20: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#
#
# Calculo auc para população 21
#
y_pred_proba = model.predict_proba(X_test_21)[:, 1]
fpr, tpr, _ = roc_curve(y_test_21, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo GERAL com pop 21: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_21, y_pred_proba)
print("IC AUC do Modelo GERAL com pop 21: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#

In [ ]:
### MODELO 20
#
# Treino do modelo
model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=5, learning_rate=0.1, verbose=0)
model.fit(X_train_20, y_train_20)
#
# Calculo auc para população GERAL
#
y_pred_proba = model.predict_proba(X_test_geral)[:, 1]
fpr, tpr, _ = roc_curve(y_test_geral, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo 20 com pop Geral: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_geral, y_pred_proba)
print("IC AUC do Modelo 20 com pop Geral: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#
# Calculo auc para população 20
#
y_pred_proba = model.predict_proba(X_test_20)[:, 1]
fpr, tpr, _ = roc_curve(y_test_20, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo 20 com pop 20: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_20, y_pred_proba)
print("IC AUC do Modelo 20 com pop 20: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#
# Calculo auc para população 21
#
y_pred_proba = model.predict_proba(X_test_21)[:, 1]
fpr, tpr, _ = roc_curve(y_test_21, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo 20 com pop 21: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_21, y_pred_proba)
print("IC AUC do Modelo 20 com pop 21: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))

In [ ]:
### MODELO 21
#
# Treino do modelo
model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=5, learning_rate=0.1, verbose=0)
model.fit(X_train_21, y_train_21)
#
# Calculo auc para população GERAL
#
y_pred_proba = model.predict_proba(X_test_geral)[:, 1]
fpr, tpr, _ = roc_curve(y_test_geral, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo 21 com pop Geral: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_geral, y_pred_proba)
print("IC AUC do Modelo 21 com pop Geral: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#
# Calculo auc para população 20
#
y_pred_proba = model.predict_proba(X_test_20)[:, 1]
fpr, tpr, _ = roc_curve(y_test_20, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo 21 com pop 20: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_20, y_pred_proba)
print("IC AUC do Modelo 21 com pop 20: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))
#
# Calculo auc para população 21
#
y_pred_proba = model.predict_proba(X_test_21)[:, 1]
fpr, tpr, _ = roc_curve(y_test_21, y_pred_proba)
roc_auc = auc(fpr, tpr)
print('AUC do Modelo 21 com pop 21: ',  roc_auc)
confidence_lower, confidence_upper = ic(y_test_21, y_pred_proba)
print("IC AUC do Modelo 21 com pop 21: [{:0.3f} - {:0.3f}]".format(confidence_lower, confidence_upper))

In [ ]:
### Gráfico
#
# Carga dos valores

data = {
    'Nome': ['Pop Ger', 'Pop 20', 'Pop 21', 
             'Pop Ger', 'Pop 20', 'Pop 21', 
             'Pop Ger', 'Pop 20', 'Pop 21'],
    'AUC': [0.742, 0.791, 0.732, 
            0.727, 0.792, 0.715, 
            0.734, 0.779, 0.732],
    'IC inferior': [0.739, 0.782, 0.725, 
                    0.724, 0.782, 0.709, 
                    0.731, 0.770, 0.725],
    'IC Superior': [0.745, 0.801, 0.740, 
                    0.730, 0.801, 0.723, 
                    0.737, 0.789, 0.740]
}

GRAFICO = pd.DataFrame(data)

# Cria uma nova coluna para os grupos, facilitando a separação visual no gráfico
GRAFICO['Grupo'] = ['Grupo 1', 'Grupo 1', 'Grupo 1', 'Grupo 2', 'Grupo 2', 'Grupo 2', 'Grupo 3', 'Grupo 3', 'Grupo 3']

# Mapeia cada grupo para uma posição no eixo x
grupo_posicoes = {'Grupo 1': 1, 'Grupo 2': 2, 'Grupo 3': 3}
GRAFICO['Posicao'] = GRAFICO['Grupo'].map(grupo_posicoes)

fig, ax = plt.subplots(figsize=(10, 6))

# Cores para as barras
cores = ['darkgreen', 'seagreen', 'olive'] 

# Distância entre as barras dentro de um mesmo grupo
largura_barra = 0.25

for i, (nome, auc, ic_inf, ic_sup, grupo, posicao) in enumerate(GRAFICO.values):
    # Calcula a posição x da barra atual
    x_pos = posicao + (i % 3 - 1) * largura_barra
    
    # Desenha a barra
    ax.bar(x_pos, auc, width=largura_barra, color=cores[i % 3], label=nome if i < 3 else "", yerr=[[auc-ic_inf], [ic_sup-auc]], capsize=5)
    
    # Adiciona o texto de AUC acima de cada barra
    ax.text(x_pos, auc+0.01, f"{auc:.3f}", ha='center', va='bottom')

# Personalizações adicionais
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Modelo Geral', 'Modelo 20', 'Modelo 21'])
ax.set_ylabel('AUC')
ax.set_title('AUC e Intervalo de Confiança dos Modelos')
ax.set_ylim(0.7, 0.81) 
ax.legend()

plt.tight_layout()
plt.show()